Попробуем снижение размерности PCA на пространстве всех признаков. Попробуем использовать различные модели. Выберем из них наилучшую.

In [1]:
import numpy as np

In [2]:
train_set = np.load('train_set.npy')
test_set = np.load('test_set.npy')

In [3]:
X_train = train_set[:, :-1]
y_train = train_set[:, -1].astype(int)

X_test = test_set[:, :-1]
y_test = test_set[:, -1].astype(int)

In [4]:
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

In [5]:
from sklearn.preprocessing import StandardScaler

In [6]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Проверим все модели для подпространств признаков.

In [7]:
from sklearn.pipeline import Pipeline

from sklearn.decomposition import PCA

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

In [8]:
counts = {'mfcc': 26, 'lpc': 10, 'formants': 13}

mfcc_slice = slice(0, counts['mfcc'])
lpc_slice = slice(counts['mfcc'], counts['mfcc'] + counts['lpc'])
formants_slice = slice(counts['mfcc'] + counts['lpc'], counts['mfcc'] + counts['lpc'] + counts['formants'])
whole_slice = slice(0, counts['mfcc'] + counts['lpc'] + counts['formants'])

slices = {'mfcc': mfcc_slice, 'lpc': lpc_slice, 'formants': formants_slice, 'whole': whole_slice}

In [9]:
models = [KNeighborsClassifier(n_neighbors=3),
          LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=1000),
          GaussianNB(),
          SVC(gamma='auto'),
          MLPClassifier(hidden_layer_sizes=(10, 10), max_iter=1000),
          DecisionTreeClassifier(min_samples_leaf=2),
          RandomForestClassifier(min_samples_leaf=2, n_estimators=20)
         ]

In [10]:
for name, feat_slice in slices.items():
    
    print('Исследование подмножества признаков', name)
    
    feature_count = X_train[:, feat_slice].shape[1]
    n_components = np.unique(list(np.arange(5, feature_count + 1, 5, dtype=int)) + [feature_count])
    
    for model in models:
        
        print(model.__class__.__name__)
        
        pca = PCA()
        pipe = Pipeline(steps=[('pca', pca), ('model', model)])
        estimator = GridSearchCV(pipe, dict(pca__n_components=n_components), cv=3)
        estimator.fit(X_train[:, feat_slice], y_train)

        print('Оптимальные параметры:', estimator.best_params_)
        print('Точность на тестовой выборке:', estimator.best_estimator_.score(X_test[:, feat_slice], y_test))
        print()

Исследование подмножества признаков mfcc
KNeighborsClassifier
Оптимальные параметры: {'pca__n_components': 26}
Точность на тестовой выборке: 0.7762295081967213

LogisticRegression
Оптимальные параметры: {'pca__n_components': 26}
Точность на тестовой выборке: 0.8885245901639345

GaussianNB
Оптимальные параметры: {'pca__n_components': 5}
Точность на тестовой выборке: 0.330327868852459

SVC
Оптимальные параметры: {'pca__n_components': 26}
Точность на тестовой выборке: 0.8729508196721312

MLPClassifier
Оптимальные параметры: {'pca__n_components': 25}
Точность на тестовой выборке: 0.4704918032786885

DecisionTreeClassifier
Оптимальные параметры: {'pca__n_components': 10}
Точность на тестовой выборке: 0.24262295081967214

RandomForestClassifier
Оптимальные параметры: {'pca__n_components': 26}
Точность на тестовой выборке: 0.5229508196721312

Исследование подмножества признаков lpc
KNeighborsClassifier
Оптимальные параметры: {'pca__n_components': 10}
Точность на тестовой выборке: 0.2106557377

C:\Users\stas\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\stas\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\stas\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Оптимальные параметры: {'pca__n_components': 10}
Точность на тестовой выборке: 0.3163934426229508

DecisionTreeClassifier
Оптимальные параметры: {'pca__n_components': 10}
Точность на тестовой выборке: 0.16065573770491803

RandomForestClassifier
Оптимальные параметры: {'pca__n_components': 10}
Точность на тестовой выборке: 0.2860655737704918

Исследование подмножества признаков formants
KNeighborsClassifier
Оптимальные параметры: {'pca__n_components': 13}
Точность на тестовой выборке: 0.04836065573770492

LogisticRegression
Оптимальные параметры: {'pca__n_components': 10}
Точность на тестовой выборке: 0.029508196721311476

GaussianNB
Оптимальные параметры: {'pca__n_components': 10}
Точность на тестовой выборке: 0.054098360655737705

SVC
Оптимальные параметры: {'pca__n_components': 10}
Точность на тестовой выборке: 0.03524590163934426

MLPClassifier


C:\Users\stas\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\stas\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\stas\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\stas\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, Converge

Оптимальные параметры: {'pca__n_components': 13}
Точность на тестовой выборке: 0.06393442622950819

DecisionTreeClassifier
Оптимальные параметры: {'pca__n_components': 13}
Точность на тестовой выборке: 0.031967213114754096

RandomForestClassifier
Оптимальные параметры: {'pca__n_components': 13}
Точность на тестовой выборке: 0.0680327868852459

Исследование подмножества признаков whole
KNeighborsClassifier
Оптимальные параметры: {'pca__n_components': 45}
Точность на тестовой выборке: 0.7557377049180328

LogisticRegression
Оптимальные параметры: {'pca__n_components': 49}
Точность на тестовой выборке: 0.8901639344262295

GaussianNB
Оптимальные параметры: {'pca__n_components': 5}
Точность на тестовой выборке: 0.2934426229508197

SVC
Оптимальные параметры: {'pca__n_components': 40}
Точность на тестовой выборке: 0.8491803278688524

MLPClassifier
Оптимальные параметры: {'pca__n_components': 45}
Точность на тестовой выборке: 0.4598360655737705

DecisionTreeClassifier
Оптимальные параметры: {'p

__Выводы__
1. Самые сильные признаки - MFCC. Только на них получается точность, как на всей совокупности признаков. Лидеры - логистическая регрессия и SVM. Точность порядка 88%. Недостаок SVM - очень долгое обучение. kNN показало тоже неплохой результат, и можно попытаться оптимизировать.
2. Признаки LPC одинаково плохо сработали. Все методы показали низкую точность классификации.
3. Признаки на основе формант показали чрезвычайно низкую точность классификации. Они не подходят для решения задачи голосовой идентификации. По крайней мере сами по себе.
4. Совокупность всех признаков позволяет чуть улучшить результат классификации. Лидер снова логистическая регрессия (89%). kNN и SVM тоже кандидаты на дальнейшую оптимизацию.